<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GPT_finetunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/docs/guides/fine-tuning

In [ ]:
!pip install colab-env --upgrade -q
!pip install openai -q

In [3]:
import colab_env
import os

from openai import OpenAI
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))


Mounted at /content/gdrive


In [ ]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

In [5]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "Summarize content you are provided with for a second-grade student."
    },
    {
      "role": "user",
      "content": "Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient civilizations since before recorded history. It is named after the Roman god Jupiter.[19] When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows,[20] and is on average the third-brightest natural object in the night sky after the Moon and Venus."
    }
  ],
  temperature=0.7,
  max_tokens=64,
  top_p=1
)

In [6]:
print(response.choices[0].message.content)

Jupiter is a big planet that is the fifth planet from the Sun. It is the largest planet in our Solar System and is made of gas. Jupiter is so big that it has a mass that is two-and-a-half times bigger than all the other planets combined. It is very bright in the sky and has been


In [7]:
#get SQuAD v2
#!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
#!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [8]:
from openai import OpenAI
client = OpenAI()

# Upload the training file
file_response = client.files.create(
  file=open("/content/gdrive/MyDrive/datasets/dataft.json", "rb"),
  purpose="fine-tune"
)

# Get the file ID
file_id = file_response.id

# Start the fine-tuning job using the file ID
cftjc=client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)

training_file=cftjc.training_file
fine_tuning_job=cftjc.id

In [9]:
# List 10 fine-tuning jobs
#client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
#client.fine_tuning.jobs.retrieve(fine_tuning_job)

# Cancel a job
#client.fine_tuning.jobs.cancel("ftjob-HHsRv4STcKEaAywwrYDjL4zI")

# List up to 10 events from a fine-tuning job
#client.fine_tuning.jobs.list_events(fine_tuning_job_id=fine_tuning_job, limit=10)

# Delete a fine-tuned model (must be an owner of the org the model was created in)
#client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

In [10]:
actual_job_id = fine_tuning_job  # Replace with a real ID
fine_tuning_job_retrieve=client.fine_tuning.jobs.retrieve(actual_job_id)
#client.fine_tuning.jobs.retrieve(actual_job_id)

In [11]:
organization_id=fine_tuning_job_retrieve.organization_id
fine_tuned_model=fine_tuning_job_retrieve.fine_tuned_model
print(fine_tuned_model)

None


In [12]:
while fine_tuned_model == None:
       fine_tuning_job_retrieve=client.fine_tuning.jobs.retrieve(actual_job_id)
       organization_id=fine_tuning_job_retrieve.organization_id
       fine_tuned_model=fine_tuning_job_retrieve.fine_tuned_model


In [13]:
print(fine_tuned_model)

ft:gpt-3.5-turbo-0125:personal::9WIPcRKC


In [14]:
from openai import OpenAI
client = OpenAI()


completion = client.chat.completions.create(
  model=fine_tuned_model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I help you today?', role='assistant', function_call=None, tool_calls=None)
